<a href="https://colab.research.google.com/github/broadinstitute/delphy/blob/main/tutorials/delphy_workflow_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
    <h1 style="font-size: 2.5em; color: #2C3E50;">Running <a href="https://github.com/broadinstitute/delphy" target="_blank">Delphy</a> is as simple as 1, 2, 3 – Ebolavirus Example</h1>
    <h2 style="font-size: 1.5em; color: #34495E;">Phylogenetic Tree Generation with Delphy on Ebolavirus Sequences</h2>
</div>

This notebook demonstrates how to generate a phylogenetic tree using [Delphy](https://github.com/broadinstitute/delphy) on viral sequences obtained from the [NCBI Virus Database](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/). Optionally, you can also upload your own sequences to be included in the analysis.

We will utilize the following tools:
- [**gget**](https://pachterlab.github.io/gget/en/virus.html) to download viral sequences from the [NCBI Virus Database](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/)
- [**MAFFT**](https://mafft.cbrc.jp/alignment/server/index.html) for creating a Multiple-Sequence Alignment (MSA)
- [**Delphy**](https://github.com/broadinstitute/delphy) to generate the phylogenetic tree

In this example notebook, we already filled out the NCBI Virus filters and options below to download and analyze all **complete *Zaire ebolavirus* sequences from *Homo sapiens* hosts collected between January 01, 2014 and June 30, 2014**. All generated files will be saved in the **results/** folder, unless you specify a different output folder below.  

Simply **click `Runtime` -> `Run all`** at the top to run this example notebook.

If you encounter any problems or questions while using this notebook, please [report them here](https://github.com/broadinstitute/delphy/issues).

Total runtime: ~4 min
___
___

# 1. Apply filters to download sequences from [NCBI Virus](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/)

In [1]:
#@title gget virus filtering options:
#@markdown Set any non-boolean filter to `None` to disable it.

def arg_str_to_bool(arg):
    if arg == "True":
        return True
    elif arg == "False":
        return False
    elif arg == "None" or arg == "":
        return None
    else:
        return arg

# -------------------------
# Virus query
# -------------------------
#@markdown ## **Virus**

virus = "Zaire ebolavirus"  #@param {type:"string"}
#@markdown  - Examples: 'Mammarenavirus lassaense' or 'coronaviridae' or 'NC_045512.2' or '142786' (Norovirus taxid).
virus = arg_str_to_bool(virus)

is_accession = False  #@param {type:"boolean"}
#@markdown  - Check this box if `virus` is given as an NCBI accession (starts with 'NC_').
#@markdown  - For SARS-CoV-2 / Influenza A optimized downloads also check the corresponding box below (see `is_sars_cov2` / `is_alphainfluenza` below)
is_accession = arg_str_to_bool(is_accession)

download_all_accessions = False  #@param {type:"boolean"}
#@markdown  - ⚠️ Downloads ALL virus accessions from NCBI (very large database).
download_all_accessions = arg_str_to_bool(download_all_accessions)

# -------------------------
# Host filters
# -------------------------
#@markdown ## **Host**

host = "Homo sapiens"  #@param {type:"string"}
#@markdown  - Host organism name OR NCBI Taxonomy ID (e.g., 'human', 'Aedes aegypti', '1335626').
#@markdown  - Input 'None' to disable filtering by host.
host = arg_str_to_bool(host)

# -------------------------
# Sequence & gene filters
# -------------------------
#@markdown ## **Sequence & gene filters**

annotated = "None"  #@param ["True", "False", "None"]
#@markdown  - True: only annotated sequences. False: only NOT annotated. None: no filter.
annotated = arg_str_to_bool(annotated)

refseq_only = False  #@param {type:"boolean"}
#@markdown  - Limit search to RefSeq genomes only (curated).
refseq_only = arg_str_to_bool(refseq_only)

nuc_completeness = "None"  #@param ["None", "complete", "partial"]
#@markdown  - Set to 'complete' to only return full-length viral genomes; set to 'partial' to only return sequences that are NOT full-length viral genomes.
nuc_completeness = arg_str_to_bool(nuc_completeness)

min_seq_length = None  #@param {type:"raw"}
max_seq_length = None  #@param {type:"raw"}

min_seq_length = arg_str_to_bool(min_seq_length)
max_seq_length = arg_str_to_bool(max_seq_length)

#@markdown
max_ambiguous_chars = None  #@param {type:"raw"}
#@markdown  - Max number of ambiguous nucleotide characters (N's).
max_ambiguous_chars = arg_str_to_bool(max_ambiguous_chars)

min_gene_count = None  #@param {type:"raw"}
max_gene_count = None  #@param {type:"raw"}
#@markdown
min_gene_count = arg_str_to_bool(min_gene_count)
max_gene_count = arg_str_to_bool(max_gene_count)
#@markdown
min_protein_count = None  #@param {type:"raw"}
max_protein_count = None  #@param {type:"raw"}
#@markdown
min_protein_count = arg_str_to_bool(min_protein_count)
max_protein_count = arg_str_to_bool(max_protein_count)
#@markdown
min_mature_peptide_count = None  #@param {type:"raw"}
max_mature_peptide_count = None  #@param {type:"raw"}
#@markdown
min_mature_peptide_count = arg_str_to_bool(min_mature_peptide_count)
max_mature_peptide_count = arg_str_to_bool(max_mature_peptide_count)

has_proteins = None  #@param {type:"raw"}
#@markdown  - Require specific proteins/genes (e.g. "spike") or list (e.g. ["spike","ORF1ab"]). **Include quotation marks.**
has_proteins = arg_str_to_bool(has_proteins)

proteins_complete = False  #@param {type:"boolean"}
#@markdown  - Only include sequences where all annotated proteins are complete.
proteins_complete = arg_str_to_bool(proteins_complete)

# -------------------------
# Location & submitter filters
# -------------------------
#@markdown ## **Location & submitter filters**

geographic_location = ""  #@param {type:"string"}
#@markdown  - Geographic location of sample collection (e.g. 'USA', 'Asia').
geographic_location = arg_str_to_bool(geographic_location)

submitter_country = ""  #@param {type:"string"}
#@markdown  - Country of the sequence submitter.
submitter_country = arg_str_to_bool(submitter_country)

lab_passaged = "None"  #@param ["True", "False", "None"]
#@markdown  - True: only lab-passaged. False: exclude lab-passaged. None: no filter.
lab_passaged = arg_str_to_bool(lab_passaged)

# -------------------------
# Date filters
# -------------------------
#@markdown ## **Dates**
#@markdown All dates should be supplied in YYYY-MM-DD format.

min_collection_date = "2014-01-01"  #@param {type:"string"}
max_collection_date = "2014-06-30"  #@param {type:"string"}

#@markdown
min_release_date = ""  #@param {type:"string"}
max_release_date = ""  #@param {type:"string"}

min_collection_date = arg_str_to_bool(min_collection_date)
max_collection_date = arg_str_to_bool(max_collection_date)

min_release_date = arg_str_to_bool(min_release_date)
max_release_date = arg_str_to_bool(max_release_date)

# -------------------------
# SARS-CoV-2 specific filters / optimizations
# -------------------------
#@markdown ## **SARS-CoV-2 / Influenza A optimizations**

lineage = ""  #@param {type:"string"}
#@markdown  - SARS-CoV-2 lineage filter (e.g. 'B.1.1.7'). (Only meaningful for SARS-CoV-2 queries.)
lineage = arg_str_to_bool(lineage)

is_sars_cov2 = False  #@param {type:"boolean"}
#@markdown  - Use optimized cached downloads for SARS-CoV-2.
is_sars_cov2 = arg_str_to_bool(is_sars_cov2)

is_alphainfluenza = False  #@param {type:"boolean"}
#@markdown  - Use optimized cached downloads for Influenza A virus (Alphainfluenza).
is_alphainfluenza = arg_str_to_bool(is_alphainfluenza)

# -------------------------
# Optional GenBank metadata enrichment
# -------------------------
#@markdown ## **GenBank metadata enrichment**

genbank_metadata = False  #@param {type:"boolean"}
#@markdown  - Fetch additional detailed metadata from GenBank into a separate CSV/XML/CSV dumps.
genbank_metadata = arg_str_to_bool(genbank_metadata)

genbank_batch_size = 200  #@param {type:"integer"}
#@markdown  - Batch size for GenBank metadata API requests. Larger may be faster but can time out (default: 200).
genbank_batch_size = int(genbank_batch_size)

# -------------------------
# Workflow / output options
# -------------------------
#@markdown ## **Workflow / output options**

keep_temp = False  #@param {type:"boolean"}
#@markdown  - Keep intermediate temporary files.
keep_temp = arg_str_to_bool(keep_temp)

outfolder = "results"  #@param {type:"string"}
#@markdown  - Output folder path.
outfolder = arg_str_to_bool(outfolder)

# 2. Optional: Upload a fasta file with your own sequences to add to the analysis
  **1) Click on the folder icon on the left.  
  2) Upload your file to the Google Colab server by dragging in your file (or use rightclick -> Upload).  
  3) Specify the name of your file here:**

In [2]:
#@title FASTA file containing additional sequences

fasta_file = ""  #@param {type:"string"}
#@markdown  - Example: 'my_fasta_file.fa' or 'my_fasta_file.fasta'.


In [3]:
#@title Metadata

#@markdown **Option 1: The metadata is the same for all sequences in your FASTA file**
metadata = {'Collection Date': 'YYYY-MM-DD', 'Geo Location': 'South Korea'}  #@param {type:"raw"}
#@markdown - The 'Collection Date' field is required. Optional: you can add as many additional columns as you wish, e.g. 'Geo Location': 'South Korea'.
#@markdown - NOTE: Use NCBI column names where applicable (see https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus for example column names)

#@markdown **Option 2: Input a CSV file with metadata for each sequence**
metadata_csv = ""  #@param {type:"string"}
#@markdown  - Example: 'my_metadata.csv'. This file must include at least an 'Accession' and 'Collection Date' column.
#@markdown  - NOTE: Make sure the IDs in the "Accession" column match the IDs of the sequences in the provided FASTA file
#@markdown  - NOTE: Use NCBI column names where applicable (see https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus for example column names)

# Convert empty strings to None
fasta_file = arg_str_to_bool(fasta_file)
metadata_csv = arg_str_to_bool(metadata_csv)

# 3. Optional: Specify Delphy arguments

In [4]:
#@title Delphy options:
mutation_rate = None  #@param {type:"raw"}
#@markdown  - Virus mutation rate (mutations per site per year), e.g. 0.01. If set to `None`, the mutation rate will be estimated from the generated tree.

delphy_steps = None  #@param {type:"raw"}
#@markdown  - Number of steps to run in the Delphy algorithm (default: 500,000 * number of sequences).

delphy_samples = 200 #@param {type:"integer"}
#@markdown  - Number of logging and tree updates (will log every `delphy_steps/delphy_samples` step).

delphy_release = "1.2.2"  #@param {type:"string"}
#@markdown  - Delphy version to use (see https://github.com/broadinstitute/delphy/releases).

#@title Delphy options:
threads = 2  #@param {type:"integer"}
#@markdown  - Number of threads to use for the MSA (Mafft) and phylogenetic tree (Delphy) generation.

# Select `Runtime` at the top of this notebook, then click `Run all` and lean back...
A completion message will be displayed below once the notebook has been successfully executed.  
**💡 Tip: Click on the folder icon 📁 on the left to view/download the files that are being generated.**
  
<br>

____
____
____
____

In [5]:
%%time

#@title # Generating the phylogenetic tree...

from IPython.display import display, HTML
from pathlib import Path
import subprocess
import sys
import shutil
import glob
import pandas as pd
import re
import os
from datetime import datetime

# ---------- Small UI helpers ----------
def log_message(text):
    display(HTML(f"<h3 style='color:green;margin:6px 0'>{text}</h3>"))

def log_message_sub(text):
    display(HTML(f"<p style='color:black; font-weight:normal; margin:4px 0;'>{text}</p>"))

def log_message_error(text):
    display(HTML(f"<h3 style='color:red;margin:6px 0'>{text}</h3>"))

def run_checked(cmd, **kwargs):
    """Run subprocess.run with sensible defaults and return CompletedProcess or raise."""
    default = dict(check=True, text=True, capture_output=True)
    default.update(kwargs)
    return subprocess.run(cmd, **default)

# ---------- Validate / create outfolder ----------
outfolder = Path(outfolder) if 'outfolder' in globals() else Path("delphy_out")
outfolder.mkdir(parents=True, exist_ok=True)

# ---------- 1) Install Python requirements (gget) ----------
log_message("1/5 Installing Python dependencies (gget)...")
try:
    !pip install --upgrade -q gget biopython
    import gget
    from Bio import SeqIO
    log_message_sub("Python dependencies installed.")
except Exception as e:
    log_message_error("Failed to install Python dependencies (gget).")
    print(e)
    raise

# ---------- 2) Download sequences from NCBI using gget ----------
log_message("2/5 Downloading viral sequences from NCBI (gget.virus)... This may take a few minutes.")
try:
    # The original code used many parameters assumed present in the notebook; call gget.virus with locals()
    # Build kwargs only from variables that exist in globals() and are not None.
    possible_kwargs = [
        "virus","is_accession","download_all_accessions","host",
        "nuc_completeness","min_seq_length","max_seq_length",
        "min_gene_count","max_gene_count","min_mature_peptide_count","max_mature_peptide_count",
        "min_protein_count","max_protein_count","max_ambiguous_chars","has_proteins","proteins_complete","annotated","refseq_only",
        "geographic_location","submitter_country","lab_passaged",
        "min_collection_date","max_collection_date","min_release_date","max_release_date",
        "lineage","is_sars_cov2","is_alphainfluenza",
        "genbank_metadata","genbank_batch_size","keep_temp","outfolder"
    ]
    gget_kwargs = {k: globals()[k] for k in possible_kwargs if k in globals() and globals()[k] is not None}
    # Ensure outfolder is string path for gget
    if "outfolder" not in gget_kwargs:
        gget_kwargs["outfolder"] = str(outfolder)
    else:
        gget_kwargs["outfolder"] = str(Path(gget_kwargs["outfolder"]))
    # Call gget.virus
    gget.virus(**gget_kwargs)

    # Find the newest fasta and metadata file in outfolder
    fasta_candidates = list(outfolder.glob("*.fasta")) + list(outfolder.glob("*.fa"))
    if not fasta_candidates:
        raise FileNotFoundError(f"No fasta files were created in {outfolder}")
    ncbi_fasta_file = max(fasta_candidates, key=lambda p: p.stat().st_mtime)

    metadata_candidates = list(outfolder.glob("*.csv")) + list(outfolder.glob("*.jsonl")) + list(outfolder.glob("*.tsv"))
    if not metadata_candidates:
        raise FileNotFoundError(f"No metadata files were created in {outfolder}")
    ncbi_metadata = max(metadata_candidates, key=lambda p: p.stat().st_mtime)

    log_message_sub(f"Download complete. FASTA: {ncbi_fasta_file.name}  Metadata: {ncbi_metadata.name}")
except Exception as e:
    log_message_error("Error while downloading sequences from NCBI.")
    print(e)
    raise

# ---------- 2b) If user provided FASTA/metadata, merge them ----------
input_fasta_file = Path(ncbi_fasta_file)
metadata_file = Path(ncbi_metadata)

if 'fasta_file' in globals() and fasta_file:
    log_message_sub("Merging user-provided FASTA with NCBI FASTA...")
    try:
        combined_fasta_file = outfolder / f"{'_'.join(str(virus).split())}_sequences_combined.fasta"
        with open(combined_fasta_file, "wb") as outfh:
            for src in [ncbi_fasta_file, Path(fasta_file)]:
                with open(src, "rb") as fh:
                    shutil.copyfileobj(fh, outfh)
        input_fasta_file = combined_fasta_file

        # Combine metadata
        combined_metadata_file = outfolder / f"{'_'.join(str(virus).split())}_metadata_combined.csv"
        ncbi_df = pd.read_csv(ncbi_metadata)

        if 'metadata_csv' in globals() and metadata_csv:
            user_meta_df = pd.read_csv(metadata_csv)
            comb_meta_df = pd.concat([ncbi_df, user_meta_df], ignore_index=True, sort=False)
            comb_meta_df.to_csv(combined_metadata_file, index=False)
            metadata_file = combined_metadata_file
        else:
            # If user provided `metadata` mapping and FASTA only
            headers = [record.id.split()[0] for record in SeqIO.parse(fasta_file, "fasta")]
            user_meta_df = pd.DataFrame({"accession": headers})
            if 'metadata' in globals() and isinstance(metadata, dict):
                for k, v in metadata.items():
                    # Broadcast scalar values or accept list-like
                    if hasattr(v, "__len__") and not isinstance(v, str) and len(v) == len(headers):
                        user_meta_df[k] = v
                    else:
                        user_meta_df[k] = [v] * len(headers)
            comb_meta_df = pd.concat([ncbi_df, user_meta_df], ignore_index=True, sort=False)
            comb_meta_df.to_csv(combined_metadata_file, index=False)
            metadata_file = combined_metadata_file

        log_message_sub(f"Merging complete. Combined FASTA: {combined_fasta_file.name}  Combined metadata: {combined_metadata_file.name}")
    except Exception as e:
        log_message_error("Error while merging user-provided data.")
        print(e)
        raise

# ---------- 3) Multiple Sequence Alignment (MSA) using MAFFT (Colab-friendly, --parttree for >10k) ----------
log_message("3/5 Multiple Sequence Alignment (MAFFT). This may take time depending on # sequences and threads.")

aligned_fasta_file = outfolder / f"{'_'.join(str(virus).split())}_aligned.afa"

def count_fasta_records(path: Path):
    return sum(1 for _ in SeqIO.parse(str(path), "fasta"))

# 1) Check for mafft binary
def mafft_is_available():
    return bool(shutil.which("mafft"))

# 2) If not available, attempt the same .deb download + dpkg approach you had for Colab
if not mafft_is_available():
    try:
        log_message_sub("MAFFT not found — attempting to download & install MAFFT .deb (Colab-compatible).")
        deb_path = outfolder / "mafft_7.526-1_amd64.deb"
        # download
        subprocess.run(["wget", "-q", "-O", str(deb_path),
                        "https://mafft.cbrc.jp/alignment/software/mafft_7.526-1_amd64.deb"],
                       check=True, text=True)
        # install
        subprocess.run(["dpkg", "-i", str(deb_path)], check=True, text=True)
        # If dpkg left missing deps, try apt-get -f install
        subprocess.run(["apt-get", "-y", "install", "-f"], check=True, text=True)
        if mafft_is_available():
            log_message_sub("MAFFT installed successfully.")
        else:
            log_message_error("MAFFT was not found after .deb install. Please install MAFFT manually in the Colab environment.")
            raise FileNotFoundError("mafft binary not found after .deb install")
    except subprocess.CalledProcessError as e:
        log_message_error("An error occurred while attempting to install MAFFT via .deb.")
        # print stderr/stdout if available
        try:
            print(e.stderr or e.stdout)
        except Exception:
            print(e)
        raise

# 3) Prepare MAFFT command, adding --parttree for large datasets
num_seqs = count_fasta_records(input_fasta_file)
threads = int(threads) if 'threads' in globals() and threads else 1

# Build command; place --parttree right after "mafft" (recommended)
if num_seqs > 10_000:
    log_message_sub(f"Detected {num_seqs:,} sequences — using MAFFT with --parttree optimized for large datasets.")
    mafft_cmd = ["mafft", "--parttree", "--auto", "--thread", str(threads), str(input_fasta_file)]
else:
    log_message_sub(f"Detected {num_seqs:,} sequences — using MAFFT standard --auto mode.")
    mafft_cmd = ["mafft", "--auto", "--thread", str(threads), str(input_fasta_file)]

# 4) Run MAFFT and write alignment directly to file (no capture_output conflict)
try:
    log_message_sub(f"Running: {' '.join(mafft_cmd[:6])} ... (command truncated in log)")
    with open(aligned_fasta_file, "w") as outfh:
        # IMPORTANT: do NOT use capture_output=True here; provide stdout=outfh instead
        subprocess.run(mafft_cmd, stdout=outfh, stderr=subprocess.PIPE, check=True, text=True)
    log_message_sub(f"MSA complete and saved to: {aligned_fasta_file.name}")
except subprocess.CalledProcessError as e:
    log_message_error("Error while running MAFFT for MSA. See stderr below:")
    stderr = e.stderr.decode() if isinstance(e.stderr, (bytes, bytearray)) else e.stderr
    print(stderr)
    raise
except Exception as e:
    log_message_error("Unexpected error while running MAFFT.")
    print(e)
    raise

# ---------- 4) Reformat FASTA headers to match Delphy format (accession|YYYY-MM-DD) ----------
log_message("4/5 Reformatting FASTA headers to 'accession|YYYY-MM-DD' (skipping entries without valid collection dates).")

def extract_and_format_date(date_string: str, accept_uncertain_dates: bool = False):
    """Return YYYY-MM-DD, YYYY-MM, or YYYY depending on accept_uncertain_dates. Return None if unrecognized."""
    if pd.isna(date_string):
        return None
    date_string = str(date_string).strip()
    # Normalize separators to '-'
    date_string = re.sub(r"[/.]", "-", date_string)
    # Try parsing strict YYYY-MM-DD
    full_date_re = re.compile(r"^\d{4}-\d{1,2}-\d{1,2}$")
    year_month_re = re.compile(r"^\d{4}-\d{1,2}$")
    year_re = re.compile(r"^\d{4}$")
    if full_date_re.match(date_string):
        return date_string
    if year_month_re.match(date_string):
        return date_string if accept_uncertain_dates else "EXCLUDED"
    if year_re.match(date_string):
        return date_string if accept_uncertain_dates else "EXCLUDED"
    # Try to parse more flexible formats (e.g., 'Jan 2 2020' or '2 Jan 2020') using datetime
    for fmt in ("%Y-%m-%d", "%d-%b-%Y", "%d %b %Y", "%b %d %Y", "%Y/%m/%d"):
        try:
            dt = datetime.strptime(date_string, fmt)
            return dt.strftime("%Y-%m-%d")
        except Exception:
            continue
    return None

def find_date_columns(df: pd.DataFrame):
    """Heuristic to find which column likely contains collection date and accession."""
    date_cols = [c for c in df.columns if re.search(r"date|collection|collected", c, flags=re.I)]
    acc_cols = [c for c in df.columns if re.search(r"acc(?:ession)?|id|seq_id", c, flags=re.I)]
    # Fallback sensible names
    if not date_cols and "Collection Date" in df.columns:
        date_cols = ["Collection Date"]
    if not acc_cols and "accession" in df.columns:
        acc_cols = ["accession"]
    # Return first hits or None
    return (date_cols[0] if date_cols else None, acc_cols[0] if acc_cols else None)

def update_fasta_headers(fasta_in: Path, csv_in: Path, fasta_out: Path, accept_uncertain_dates: bool = False):
    """Rewrite fasta headers to 'accession|date' using CSV metadata. Exclude entries without valid date."""
    df = pd.read_csv(csv_in, dtype=str)
    date_col, acc_col = find_date_columns(df)
    if date_col is None or acc_col is None:
        raise ValueError(f"Could not find accession/date columns in {csv_in}. Found columns: {list(df.columns)}")
    accession_to_date = pd.Series(df[date_col].values, index=df[acc_col]).to_dict()

    included = 0
    excluded = 0
    unrecognized = 0

    with open(fasta_in) as fh_in, open(fasta_out, "w") as fh_out:
        for record in SeqIO.parse(fh_in, "fasta"):
            original_accession = record.id.split()[0]
            date_val = accession_to_date.get(original_accession) or accession_to_date.get(original_accession.split("|")[0])
            formatted = extract_and_format_date(date_val, accept_uncertain_dates=accept_uncertain_dates)
            if formatted == "EXCLUDED":
                excluded += 1
                continue
            if formatted is None:
                unrecognized += 1
                continue
            # Set header and remove description
            record.id = f"{original_accession}|{formatted}"
            record.description = ""
            SeqIO.write(record, fh_out, "fasta")
            included += 1

    return {"included": included, "excluded": excluded, "unrecognized": unrecognized}

aligned_fasta_file_clean = outfolder / f"{'_'.join(str(virus).split())}_aligned_headers_adjusted.afa"
try:
    accept_uncertain_dates = bool(globals().get("accept_uncertain_dates", False))
    stats = update_fasta_headers(aligned_fasta_file, metadata_file, aligned_fasta_file_clean, accept_uncertain_dates=accept_uncertain_dates)
    log_message_sub(f"Reformatting complete. Included {stats['included']} sequences; excluded (incomplete dates) {stats['excluded']}; unrecognized {stats['unrecognized']}")
except Exception as e:
    log_message_error("Error while reformatting FASTA headers.")
    print(e)
    raise

# ---------- 5) Run Delphy ----------
log_message("5/5 Preparing and running Delphy. This step can be long depending on steps/samples/sequence count.")

def count_fasta_records(path: Path):
    return sum(1 for _ in SeqIO.parse(str(path), "fasta"))

num_seqs = count_fasta_records(aligned_fasta_file_clean)
if num_seqs == 0:
    raise RuntimeError(f"No sequences found in {aligned_fasta_file_clean}")

# sensible defaults for delphy params if not provided
delphy_steps = globals().get("delphy_steps")
delphy_samples = globals().get("delphy_samples", 100)
if delphy_steps is None:
    delphy_steps = int(500_000 * max(1, num_seqs))

log_every = int(delphy_steps / delphy_samples) if delphy_samples else max(1, delphy_steps // 100)
tree_every = log_every
delphy_snapshot_every = log_every

# Download delphy if not present
delphy_bin = (Path.cwd() / "delphy").resolve()
if not delphy_bin.exists():
    delphy_release = globals().get("delphy_release", "1.2.2")  # user may override
    tarball = outfolder / "delphy_release.tar.gz"
    try:
        download_url = f"https://github.com/broadinstitute/delphy/releases/download/{delphy_release}/delphy-linux-x86_64-ubuntu22.tar.gz"
        log_message_sub(f"Downloading Delphy release {delphy_release}.")
        run_checked(["wget", "-q", "-O", str(tarball), download_url])
        run_checked(["tar", "-xzf", str(tarball)], check=True)
        # Try to find delphy binary in extracted files
        found = list(Path(".").glob("**/delphy"))
        if found:
            src = found[0].resolve()
            dst = delphy_bin.resolve()

            # Make sure the found binary is executable
            src.chmod(src.stat().st_mode | 0o111)

            # Only copy if it's not already the same file
            if src != dst:
                shutil.copy(src, dst)
                dst.chmod(dst.stat().st_mode | 0o111)
                log_message_sub("Delphy binary obtained.")
            else:
                log_message_sub("Delphy binary already present in working directory; skipping copy.")

    except Exception as e:
        log_message_error("Failed to download/extract Delphy. Please ensure `delphy` binary is available in the working directory.")
        print(e)
        raise

# Build Delphy command
beast_log_out = outfolder / f"{'_'.join(str(virus).split())}_delphy_beast.log"
delphy_beast_tree_out = outfolder / f"{'_'.join(str(virus).split())}_delphy_beast.trees"
dphy_out = outfolder / f"{'_'.join(str(virus).split())}_delphy_out.dphy"

mutation_rate = globals().get("mutation_rate", None)
threads = int(globals().get("threads", 1))
cmd = [
    str(delphy_bin),
    "--v0-log-every", str(log_every),
    "--v0-tree-every", str(tree_every),
    "--v0-delphy-snapshot-every", str(delphy_snapshot_every),
    "--v0-threads", str(threads),
    "--v0-steps", str(delphy_steps),
    "--v0-in-fasta", str(aligned_fasta_file_clean),
    "--v0-out-log-file", str(beast_log_out),
    "--v0-out-trees-file", str(delphy_beast_tree_out),
    "--v0-out-delphy-file", str(dphy_out),
]
if mutation_rate:
    cmd = [str(delphy_bin), "--v0-fix-mutation-rate", "--v0-init-mutation-rate", str(mutation_rate)] + cmd[1:]

log_message_sub(f"Running Delphy with command: {' '.join(cmd[:8])} ... (truncated)")

try:
    # Run delphy and stream output to notebook
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    for line in proc.stdout:
        # print lines incrementally so the notebook shows progress
        print(line, end="")
    ret = proc.wait()
    if ret != 0:
        raise subprocess.CalledProcessError(ret, cmd)

    # Success
    outfolder_display = str(outfolder) if outfolder not in [None, "", "None"] else None

    if outfolder_display:
        location_text = f"Files generated in this notebook are in the left-hand file browser and in the <code>{outfolder_display}/</code> folder."
    else:
        location_text = "Files generated in this notebook are in the left-hand file browser."

    display(HTML(f"""
    <h2 style='color:green'>All done! 🎉</h2>
    <p>{location_text}</p>
    <ul>
      <li>Aligned FASTA: <code>{aligned_fasta_file_clean.name}</code></li>
      <li>Delphy output (.dphy): <code>{dphy_out.name}</code></li>
      <li>Delphy log (.log): <code>{beast_log_out.name}</code></li>
      <li>Delphy trees (.trees): <code>{delphy_beast_tree_out.name}</code></li>
    </ul>
    <p><strong>We recommend downloading the <code>.afa</code>, <code>.dphy</code>, and <code>metadata.csv</code> files.
    To visualize your Delphy output, upload the <code>.dphy</code> (and optionally <code>metadata.csv</code>) file(s) to
    <a href='https://delphy.fathom.info/' target='_blank'>https://delphy.fathom.info/</a>.</strong></p>
    """))

except subprocess.CalledProcessError as e:
    log_message_error("An error occurred while running Delphy. See captured output above for details.")
    print(e)
    raise
except Exception as e:
    log_message_error("Unexpected error while running Delphy.")
    print(e)
    raise


INFO:gget.utils:Starting virus data retrieval process...
INFO:gget.utils:Query parameters: virus='Zaire ebolavirus', is_accession=False, outfolder='results'
INFO:gget.utils:============================================================
INFO:gget.utils:STEP 1: VALIDATING INPUT ARGUMENTS AND OUTPUT DIRECTORY SETUP...
INFO:gget.utils:============================================================
INFO:gget.utils:Input validation completed successfully
INFO:gget.utils:Using specified output folder: results
INFO:gget.utils:============================================================
INFO:gget.utils:STEP 2: CHECKING FOR SARS-CoV-2 AND INFLUENZA A QUERIES TO APPLY OPTIMIZED CACHED PATHWAY
INFO:gget.utils:============================================================
INFO:gget.utils: Skipping this step. No SARS-CoV-2 query detected.
INFO:gget.utils: Skipping this step. No Alphainfluenza query detected.
INFO:gget.utils:============================================================
INFO:gget.utils:STEP 3

✓ Batch 1: Downloaded 200 sequences (3.68 MB)


INFO:gget.utils:Successfully saved 277 sequences to: results/tmp_20260213_014744_919fd7/virus_sequences_20260213_014744_919fd7.fasta (5.10 MB)
INFO:gget.utils:Downloaded FASTA file: results/tmp_20260213_014744_919fd7/virus_sequences_20260213_014744_919fd7.fasta (5.10 MB)
INFO:gget.utils:============================================================
INFO:gget.utils:STEP 6: Applying sequence-dependent filters and saving results
INFO:gget.utils:============================================================
INFO:gget.utils:No sequence-dependent filters specified, skipping this step.
INFO:gget.utils:All 277 downloaded sequences will be saved
INFO:gget.utils:============================================================
INFO:gget.utils:STEP 7: Saving final output files
INFO:gget.utils:============================================================
INFO:gget.utils:Saving 277 filtered sequences and their metadata...
INFO:gget.utils:✅ FASTA file saved: results/Zaire_ebolavirus_sequences.fasta (5.10 MB)


✓ Batch 2: Downloaded 77 sequences (1.42 MB)


# Seed: 1025141311
Reading fasta file results/Zaire_ebolavirus_aligned_headers_adjusted.afa
- read 1 sequences so far (19324 of 5346377 bytes = 0.4%)
- read 2 sequences so far (38625 of 5346377 bytes = 0.7%)
- read 3 sequences so far (57926 of 5346377 bytes = 1.1%)
- read 4 sequences so far (77227 of 5346377 bytes = 1.4%)
- read 5 sequences so far (96528 of 5346377 bytes = 1.8%)
- read 6 sequences so far (115829 of 5346377 bytes = 2.2%)
- read 7 sequences so far (135130 of 5346377 bytes = 2.5%)
- read 8 sequences so far (154431 of 5346377 bytes = 2.9%)
- read 9 sequences so far (173732 of 5346377 bytes = 3.2%)
- read 10 sequences so far (193033 of 5346377 bytes = 3.6%)
- read 11 sequences so far (212334 of 5346377 bytes = 4.0%)
- read 12 sequences so far (231635 of 5346377 bytes = 4.3%)
- read 13 sequences so far (250936 of 5346377 bytes = 4.7%)
- read 14 sequences so far (270237 of 5346377 bytes = 5.1%)
- read 15 sequences so far (289538 of 5346377 bytes = 5.4%)
- read 16 sequences so

CPU times: user 4.13 s, sys: 624 ms, total: 4.76 s
Wall time: 6min 33s
